<h1 align=center><font size = 5>Failure Prediction</font></h1>


I tried three models: logistic regression, Kmeans and NeuroNetwork.

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from keras.layers import Dropout
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from pandas.plotting import andrews_curves

Using TensorFlow backend.
/anaconda3/envs/python37/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/python37/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/python37/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/python37/lib/python3.6/site-packag

In [2]:
# read csv file to a dataframe
df = pd.read_csv('predictive_maintenance_case.csv')
df.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,1/1/15,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,1/1/15,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,1/1/15,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,1/1/15,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,1/1/15,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


### Preprocessing
In this part, I turned the 'date' and 'device' column into int, checked the correlation and select features. And then I scaled the values and split them into training and testing sets.

In [3]:
# turn 'date' into int
days = lambda x : (pd.to_datetime(x)-pd.to_datetime('1/1/15')).dt.days
df[['date']] = df[['date']].apply(days)
# turn 'device' into onehot-vector
df = df.join(pd.get_dummies(df.device,prefix='device'))
df = df.drop('device',axis=1)
df.head()

,date,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,...,device_Z1F1HSWK,device_Z1F1Q9BD,device_Z1F1R76A,device_Z1F1RE71,device_Z1F1RJFA,device_Z1F1VMZB,device_Z1F1VQFY,device_Z1F26YZB,device_Z1F282ZV,device_Z1F2PBHX
0,0,0,215630672,56,0,52,6,407438,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,61370680,0,3,0,6,403174,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,173295968,0,0,0,12,237394,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,79694024,0,0,0,6,410186,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,135970480,0,0,0,15,313173,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# check correlation and delete redundant features
temp=[]
columns=[]
for column in df.columns.values.tolist():
    pearson_coef, p_value = stats.pearsonr(df[column], df['failure'])
    temp.append(pearson_coef)
    if pearson_coef not in temp[:-1]:
        print(column,pearson_coef)
        columns.append(column)

date 0.0006265146007184932
failure 1.0
metric1 0.001983484281090146
metric2 0.05290157991458708
metric3 -0.0009484301869248991
metric4 0.06739847485574071
metric5 0.0022697308795808506
metric6 -0.0005503238387569551
metric7 0.11905458506084267
metric9 0.0016215740167767306
device_S1F01085 -0.00020266351485579264
device_S1F013BB -0.0002026635148557925
device_S1F01E6Y -0.0005733157039486517
device_S1F01R2B -0.001236605646252956
device_S1F01XDJ -0.0008521722352638513
device_S1F023H2 0.021933358369154696
device_S1F02A0J -0.0012476670671129762
device_S1F02L38 -0.0007675183395772688
device_S1F03RV3 -0.00018500488805468844
device_S1F03YZM 0.005418483483036847
device_S1F044ET -0.0012722062126517737
device_S1F04DH8 -0.00041371674039145204
device_S1F04KSC -0.00016547273773747547
device_S1F06ZX2 -0.0008759794815026017
device_S1F09DZQ 0.0057256526279989475
device_S1F0AADQ -0.0013634754696388322
device_S1F0BVK1 -0.0012421485805738522
device_S1F0BWZ3 -0.0013834987463145522
device_S1F0C95J -0.0014032

In [5]:
# choose features: There is no feature shows high relavance with 'failure', so I choose the following features. 
columns=['date','failure','metric1','metric2','metric3','metric4','metric5','metric6','metric7','metric9']


In [6]:
# normalize and split the dataset
X = df[columns]
X = X.drop(['failure'],axis=1).values
y = df[['failure']].values
y = y.reshape(1,len(y))[0]

# normalize data
# X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
# print(X[0:5])
X = preprocessing.maxabs_scale(X)
print(X)

#split into train and test data
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2) #random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

[[0.00000000e+00 8.83223757e-01 8.61962812e-04 ... 5.91208731e-01
  0.00000000e+00 3.74311534e-04]
 [0.00000000e+00 2.51374455e-01 0.00000000e+00 ... 5.85021497e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.09820707e-01 0.00000000e+00 ... 3.44468129e-01
  0.00000000e+00 0.00000000e+00]
 ...
 [1.00000000e+00 7.79433218e-02 7.43750770e-02 ... 5.08458836e-01
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 9.29601711e-01 0.00000000e+00 ... 5.20894247e-01
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 7.19783954e-02 0.00000000e+00 ... 5.09940348e-01
  0.00000000e+00 0.00000000e+00]]
Train set: (99595, 9) (99595,)
Test set: (24899, 9) (24899,)


### Functions
These two functions are used to compute precision, recall, f1-score, TP, TN, FP, FN

In [9]:
# compute precision and recall
def cal_p_r(y_test,yhat):
     print(classification_report(y_test, yhat))

In [10]:
# compute TP, TN, FP, FN
def cal_base(y_true, y_pred):
    y_pred_positive = np.round(np.clip(y_pred, 0, 1))
    y_pred_negative = 1 - y_pred_positive

    y_positive = np.round(np.clip(y_true, 0, 1))
    y_negative = 1 - y_positive

    TP = np.sum(y_positive * y_pred_positive)
    TN = np.sum(y_negative * y_pred_negative)

    FP = np.sum(y_negative * y_pred_positive)
    FN = np.sum(y_positive * y_pred_negative)

    print('TP={}, TN={}, FP={}, FN={}'.format(TP, TN, FP, FN))

### Models
### 1.logistic regression

In [11]:
# create the model
LR = LogisticRegression(penalty='l2',C=0.1, solver='saga',class_weight='balanced')
# train the model
LR.fit(X_train,y_train)
# prediction
yhat = LR.predict(X_test)
yhat

/anaconda3/envs/python37/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


array([1, 0, 0, ..., 1, 1, 1])

In [12]:
# evaluation
cal_p_r(y_test, yhat)
cal_base(y_test, yhat)

              precision    recall  f1-score   support

           0       1.00      0.47      0.64     24882
           1       0.00      0.59      0.00        17

   micro avg       0.47      0.47      0.47     24899
   macro avg       0.50      0.53      0.32     24899
weighted avg       1.00      0.47      0.64     24899

TP=10, TN=11666, FP=13216, FN=7


This model doesn't performs well.

### 2. kmeans

In [13]:
# create the model
k_means = KMeans( n_clusters = 2, n_init = 12)
# train
k_means.fit(X_train)
# prediction
result = k_means.predict(X_test)

In [14]:
# evaluation
cal_p_r(y_test, result)
cal_base(y_test, result)

              precision    recall  f1-score   support

           0       1.00      0.50      0.67     24882
           1       0.00      0.41      0.00        17

   micro avg       0.50      0.50      0.50     24899
   macro avg       0.50      0.46      0.33     24899
weighted avg       1.00      0.50      0.66     24899

TP=7, TN=12401, FP=12481, FN=10


This model doesn't performs well either. There are too many FP and FN.

### 3. Neuro Network

In [15]:
# Create the model
model = Sequential()
model.add(Dense(12, input_dim=9, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# turn the label to 2 dimentions.
ohe = OneHotEncoder()
y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)

y_train_1=ohe.fit_transform(y_train).toarray()
y_test_1=ohe.fit_transform(y_test).toarray()

y_train_1

/anaconda3/envs/python37/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/envs/python37/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
# compute class weight and train the model
neg = sum(y_train==0)[0] 
pos = sum(y_train==1)[0]
total = len(y_train)
weight_for_0 = (1 / neg)*(total)/2.0
weight_for_1 = (1 / pos)*(total)/2.0

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='acc', 
    verbose=1,
    patience=10,
    mode='max')
#     restore_best_weights=True)

class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(X_train, y_train_1, 
                    epochs=20, 
                    batch_size=1000, 
                    callbacks = [early_stopping], 
                    validation_data=(X_test, y_test_1),
                    class_weight=class_weight)


Train on 99595 samples, validate on 24899 samples
Epoch 1/20
99595/99595 [==============================] - 0s 4us/step - loss: 0.6759 - acc: 0.9610 - val_loss: 0.5092 - val_acc: 0.9894
Epoch 2/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6525 - acc: 0.9939 - val_loss: 0.4469 - val_acc: 0.9991
Epoch 3/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6501 - acc: 0.9988 - val_loss: 0.4463 - val_acc: 0.9989
Epoch 4/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6446 - acc: 0.9985 - val_loss: 0.4102 - val_acc: 0.9989
Epoch 5/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6503 - acc: 0.9986 - val_loss: 0.4499 - val_acc: 0.9987
Epoch 6/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6422 - acc: 0.9982 - val_loss: 0.4396 - val_acc: 0.9984
Epoch 7/20
99595/99595 [==============================] - 0s 2us/step - loss: 0.6485 - acc: 0.9979 - val_loss: 0.4321 - val_acc: 0.998

In [19]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test_1)):
    test.append(np.argmax(y_test_1[i]))

In [20]:
cal_p_r(test, pred)
cal_base(test,pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     24882
           1       0.04      0.12      0.06        17

   micro avg       1.00      1.00      1.00     24899
   macro avg       0.52      0.56      0.53     24899
weighted avg       1.00      1.00      1.00     24899

TP=2, TN=24835, FP=47, FN=15


Although FP decreased a lot, only 2 failure instances are predicted correct. This is not good.

Because the data is extreamly unbalanced, I augmented (adding random noise) the failure class data and added them into the training and testing dataset. The result is better.

In [21]:
# add more data
train_add=[]
for jj in range(1000):
    for ii,value in enumerate(y_train):
        if value==1:
            train_add.append((np.random.randn(9)*0.05+1)*X_train[ii])
train_add=np.array(train_add)
print(type(train_add))
print(len(train_add))

print(X_train.shape,train_add.shape)
print(type(X_train),type(train_add))
X_train=np.concatenate((X_train,train_add),axis=0)
print(X_train.shape)


test_add=[]
for jj in range(1000):
    for ii,value in enumerate(y_test):
        if value==1:
            test_add.append((np.random.randn(9)*0.05+1)*X_test[ii])
test_add=np.array(test_add)
print(type(test_add))
print(len(test_add))

print(X_test.shape,test_add.shape)
print(type(X_test),type(test_add))
X_test=np.concatenate((X_test,test_add),axis=0)
print(X_test.shape)

y_train=np.concatenate((y_train,np.array([[1]]*sum(y_train==1)[0]*1000)),axis=0)
y_test=np.concatenate((y_test,np.array([[1]]*sum(y_test==1)[0]*1000)),axis=0)


<class 'numpy.ndarray'>
89000
(99595, 9) (89000, 9)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(188595, 9)
<class 'numpy.ndarray'>
17000
(24899, 9) (17000, 9)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(41899, 9)


In [22]:
ohe = OneHotEncoder()
y_train_2=ohe.fit_transform(y_train).toarray()
y_test_2=ohe.fit_transform(y_test).toarray()
y_train_2

/anaconda3/envs/python37/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/envs/python37/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [23]:
# Neural network
model = Sequential()

model.add(Dense(12, input_dim=9, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='acc', 
    verbose=1,
    patience=10,
    mode='max')

class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(X_train, y_train_2, 
                    epochs=20, 
                    batch_size=1000, 
                    callbacks = [early_stopping], 
                    validation_data=(X_test, y_test_2))


Train on 188595 samples, validate on 41899 samples
Epoch 1/20
188595/188595 [==============================] - 1s 3us/step - loss: 0.6889 - acc: 0.5440 - val_loss: 0.6707 - val_acc: 0.6482
Epoch 2/20
188595/188595 [==============================] - 0s 1us/step - loss: 0.6635 - acc: 0.6179 - val_loss: 0.6413 - val_acc: 0.6590
Epoch 3/20
188595/188595 [==============================] - 0s 2us/step - loss: 0.6303 - acc: 0.6465 - val_loss: 0.6017 - val_acc: 0.6785
Epoch 4/20
188595/188595 [==============================] - 0s 1us/step - loss: 0.5887 - acc: 0.7005 - val_loss: 0.5392 - val_acc: 0.7669
Epoch 5/20
188595/188595 [==============================] - 0s 1us/step - loss: 0.5435 - acc: 0.7430 - val_loss: 0.4912 - val_acc: 0.7736
Epoch 6/20
188595/188595 [==============================] - 0s 1us/step - loss: 0.5046 - acc: 0.7661 - val_loss: 0.4648 - val_acc: 0.7930
Epoch 7/20
188595/188595 [==============================] - 0s 2us/step - loss: 0.4773 - acc: 0.7779 - val_loss: 0.4422 -

In [24]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test_2)):
    test.append(np.argmax(y_test_2[i]))

In [25]:
# prediction
y_pred = model.predict(X_test)

In [26]:
cal_p_r(test, pred)
cal_base(test,pred)

              precision    recall  f1-score   support

           0       0.87      0.89      0.88     24882
           1       0.84      0.80      0.82     17017

   micro avg       0.86      0.86      0.86     41899
   macro avg       0.85      0.85      0.85     41899
weighted avg       0.85      0.86      0.85     41899

TP=13639, TN=22199, FP=2683, FN=3378


### Summary
From the precision, recall, TP, TN, FP, FN values we can see that 
1. Logistic regration and kmeans can not predict the failure well.
2. Neural network is better than them.
3. After adding augmented data, the nerual network performs better. If we assume that the future unseen failure data are similar to the augmented failure data, we can use this model to do the prediction.